
##### refunder stream

this notebook starts a stream to score completed orders for potential refunds

In [0]:
%pip install --upgrade databricks-sdk

In [0]:
#### TODO: for some reason the experiments tracing page is not registering the calls made through this, investigate that
###         update: i believe this happens because the experiment created by the task of this job (see ./jobs/refund_recommender_stream) comes from a "notebook" that is ephemeral (exists only for the scope of the job) and so inference calls outside the runtime of the job go nowhere

In [ ]:
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.jobs as j
import os, json

w = WorkspaceClient()

# Resolve the workspace notebook path (keeps your approach)
notebook_abs_path = os.path.abspath("../jobs/refund_recommender_stream")
notebook_dbx_path = notebook_abs_path.replace(
    os.environ.get("DATABRICKS_WORKSPACE_ROOT", "/Workspace"),
    "/Workspace"
)

job = w.jobs.create(
    name="Refund Recommender Stream",
    schedule=j.CronSchedule(
        quartz_cron_expression="0 0/10 * * * ?",
        timezone_id="UTC",
        pause_status=j.PauseStatus.UNPAUSED),
    tasks=[
        j.Task(
            task_key="refund_recommender_stream",
            notebook_task=j.NotebookTask(
                notebook_path=notebook_dbx_path,
                base_parameters={
                    "CATALOG": dbutils.widgets.get("CATALOG"),
                    "REFUND_AGENT_ENDPOINT_NAME": dbutils.widgets.get("REFUND_AGENT_ENDPOINT_NAME")
                },
            )
        )
    ],
)
print(f"Created job_id={job.job_id}")

import sys
sys.path.append('../utils')
from uc_state import add

add(dbutils.widgets.get("CATALOG"), "jobs", job)

w.jobs.run_now(job_id=job.job_id)
